###### **1º Passo: Configurações Iniciais**

In [1]:
import ssl
import pandas as pd
import os
from notebookutils import mssparkutils

# Ignorar erro de SSL para conseguir baixar do site do governo
ssl._create_default_https_context = ssl._create_unverified_context

StatementMeta(, cce442e4-47df-411a-afd1-7f62f3b5f21b, 3, Finished, Available, Finished)

In [2]:
raw_dados_publicos_abfss_files = "abfss://3ed9004c-a27b-47fb-8a19-3a0a8508fbf3@onelake.dfs.fabric.microsoft.com/34b49a7f-cf79-410e-8c7a-2b00f7d44771/Files"

StatementMeta(, cce442e4-47df-411a-afd1-7f62f3b5f21b, 4, Finished, Available, Finished)

In [3]:
pasta_armazenamento = "Mdic_Dimensoes"

caminho_destino = f"{raw_dados_publicos_abfss_files}/{pasta_armazenamento}"

mssparkutils.fs.mkdirs(caminho_destino)

StatementMeta(, cce442e4-47df-411a-afd1-7f62f3b5f21b, 5, Finished, Available, Finished)

True

###### **2º Passo: Lista com todos os links de dimensões que serão armazenados**

In [21]:
# Lista com todos os links fornecidos
urls = [
    "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_SH.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CUCI.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_ISIC.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/ISIC_CUCI.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CGCE.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_FAT_AGREG.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_PPE.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_PPI.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_UNIDADE.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS_BLOCO.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/UF_MUN.csv",
    "https://balanca.economia.gov.br/balanca/bd/tabelas/VIA.csv"
]

print(f"Total de arquivos para processar: {len(urls)}\n")

StatementMeta(, 3c1990d5-0bf3-4a36-a3f6-09640b60d943, 23, Finished, Available, Finished)

Total de arquivos para processar: 14



###### **3º Passo: Loop para processamento dos dados**

In [ ]:
for url in urls:
    # Extrai o nome do arquivo
    nome_arquivo_base = url.split("/")[-1].replace(".csv", "")
    nome_parquet_final = f"{nome_arquivo_base}.parquet"
    
    print(f">>> [⬇️] Processando: {nome_arquivo_base}...")
    
    try:
        # A. Leitura
        pdf = pd.read_csv(url, sep=';', encoding='latin1')
        df_spark = spark.createDataFrame(pdf.astype(str))
        
        # Usamos 'caminho_destino' como base fixa, sem sobrescrevê-la
        
        # Caminho temporário para escrita do Spark
        caminho_temp = f"{caminho_destino}/temp_{nome_arquivo_base}"
        
        # Caminho final onde o arquivo vai ficar
        caminho_arquivo_final = f"{caminho_destino}/{nome_parquet_final}"
        
        # Escrita na pasta temporária
        df_spark.coalesce(1).write.mode("overwrite").format("parquet").save(caminho_temp)
            
        # D. Renomear e Mover
        arquivos_temp = mssparkutils.fs.ls(caminho_temp)
        arquivo_encontrado = False
        
        for arquivo in arquivos_temp:
            if arquivo.name.startswith("part-") and arquivo.name.endswith(".parquet"):
                
                # Se o arquivo final já existe, deleta ele antes de mover o novo
                try:
                    mssparkutils.fs.rm(caminho_arquivo_final, False) 
                except:
                    pass 

                # Move do temp para o nome final
                mssparkutils.fs.mv(arquivo.path, caminho_arquivo_final)
                arquivo_encontrado = True
                break
        
        # E. Limpeza da pasta temporária
        mssparkutils.fs.rm(caminho_temp, True)
        
        if arquivo_encontrado:
            print(f"    Finalizado [✅]")
        else:
            print(f"    [❌] Arquivo parquet não gerado corretamente.")

    except Exception as e:
        print(f"    [❌] Erro ao processar {nome_arquivo_base}: {str(e)}")

    print("-" * 30)

print("\nProcessamento em massa finalizado!")